In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random

import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from helper_functions_2 import (
    convert_bytes,
    convert_prefetchdataset_to_numpy_arrays,
    evaluate_time_of_prediction,
    get_file_size,
    predict_and_print_full_results,
)

2024-07-28 22:24:08.666982: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 22:24:08.917640: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 22:24:08.917697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 22:24:08.981740: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 22:24:09.088700: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 22:24:09.090458: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
sample_rate = 16000

In [4]:
train_dataset = tf.keras.utils.audio_dataset_from_directory(
    "dataset/training",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)
test_dataset = tf.keras.utils.audio_dataset_from_directory(
    "dataset/testing",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)
val_dataset = tf.keras.utils.audio_dataset_from_directory(
    "dataset/validation",
    labels="inferred",
    sampling_rate=16000,
    batch_size=32,
    shuffle=True,
    seed=3407,
)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-07-28 22:24:13.759487: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-28 22:24:13.760025: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-07-28 22:24:14.003025: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-07-28 22:24:14.074072: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [5]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(
    train_dataset, data_type="time-series"
)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(
    val_dataset, data_type="time-series"
)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(
    test_dataset, data_type="time-series"
)

In [6]:
BASE_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_baseline.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48000, 1)]           0         []                            
                                                                                                  
 conv1 (Conv1D)              (None, 24000, 28)            224       ['input_1[0][0]']             
                                                                                                  
 maxpool1 (MaxPooling1D)     (None, 11999, 28)            0         ['conv1[0][0]']               
                                                                                                  
 fire2_squeeze (Conv1D)      (None, 11999, 4)             116       ['maxpool1[0][0]']            
                                                                                              

### Convert to TF Lite format

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp1hw8dyck/assets


INFO:tensorflow:Assets written to: /tmp/tmp1hw8dyck/assets
2024-07-28 22:25:01.258436: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 22:25:01.258530: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 22:25:01.259130: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp1hw8dyck
2024-07-28 22:25:01.265778: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 22:25:01.265816: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp1hw8dyck
2024-07-28 22:25:01.279355: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-28 22:25:01.286673: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 22:25:01.528815: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [8]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    tflite_model, x_val_np, y_val_np, model_format="tf_lite"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    tflite_model, x_test_np, y_test_np, model_format="tf_lite"
)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 91.95%
Precision: 97.75%
F1-score: 94.76%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9461470006413389

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9490614960562602

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9486317826775494

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9482568799678605

Time for Test dataset:
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%

Time to make a prediction for a single data point
Mean: 0.013 seconds
Max: 0.023 seconds
Min: 0.012 seconds



In [9]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 150.871 Kilobytes


In [10]:
len(x_test_np)

1393

### Dynamic range quantization

In [11]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpmfvx14l2/assets


INFO:tensorflow:Assets written to: /tmp/tmpmfvx14l2/assets
2024-07-28 22:32:31.593850: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 22:32:31.593921: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 22:32:31.594122: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpmfvx14l2
2024-07-28 22:32:31.600215: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 22:32:31.600290: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpmfvx14l2
2024-07-28 22:32:31.620102: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 22:32:31.819613: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpmfvx14l2
2024-07-28 22:32:31.881367: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [12]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(
    dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

DRQ_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 91.95%
Precision: 97.75%
F1-score: 94.76%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9461470006413389

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9443041864472227

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9486317826775494

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9500742705036384

Time for Test dataset:
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%

Time to make a prediction for a single data point
Mean: 0.021 seconds
Max: 0.037 seconds


In [13]:
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

File size: 89.656 Kilobytes


### Float16 quantization

In [14]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp0cc9t2dp/assets


INFO:tensorflow:Assets written to: /tmp/tmp0cc9t2dp/assets
2024-07-28 22:40:48.120245: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 22:40:48.120322: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 22:40:48.120531: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp0cc9t2dp
2024-07-28 22:40:48.126885: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 22:40:48.126923: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp0cc9t2dp
2024-07-28 22:40:48.145350: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 22:40:48.355671: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp0cc9t2dp
2024-07-28 22:40:48.418621: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [15]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    float16_quant_model, x_val_np, y_val_np, model_format="tf_lite"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    float16_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(
    float16_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

FLOAT16_QUANT_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.52%
Recall: 91.95%
Precision: 97.75%
F1-score: 94.76%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9461470006413389

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9472879754441966

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9486317826775494

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9457113355907262

Time for Test dataset:
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%

Time to make a prediction for a single data point
Mean: 0.017 seconds
Max: 0.031 seconds


In [16]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 97.117 Kilobytes


### Full integer quantization

In [17]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_test_np).batch(1).take(500):
        # Model has only one input so each data point has one element.
        yield [input_value]


full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

full_int_quant_model = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp9y3wqglv/assets


INFO:tensorflow:Assets written to: /tmp/tmp9y3wqglv/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-28 22:45:49.082662: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 22:45:49.082772: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 22:45:49.083090: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp9y3wqglv
2024-07-28 22:45:49.089875: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 22:45:49.089905: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp9y3wqglv
2024-07-28 22:45:49.109540: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 22:45

In [18]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    full_int_quant_model,
    x_val_np,
    y_val_np,
    model_format="tf_lite",
    input_data_uint8_type=True,
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    full_int_quant_model,
    x_test_np,
    y_test_np,
    model_format="tf_lite",
    input_data_uint8_type=True,
)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(
    full_int_quant_model,
    x_test_np,
    y_test_np,
    model_format="tf_lite",
    input_data_uint8_type=True,
)

FULL_INT_QUANT_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_full_int_q.tflite"
open(FULL_INT_QUANT_MODEL_FILE_NAME, "wb").write(full_int_quant_model)
print("\n")
print("Model file name: ", FULL_INT_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FULL_INT_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:


KeyboardInterrupt: 

### Float fallback

In [20]:
float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(
    baseline_model
)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpybb6rhno/assets


INFO:tensorflow:Assets written to: /tmp/tmpybb6rhno/assets
/home/polina/newname/venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-06-18 23:41:27.086769: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-18 23:41:27.086822: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-18 23:41:27.087102: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpybb6rhno
2024-06-18 23:41:27.095430: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-18 23:41:27.095515: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpybb6rhno
2024-06-18 23:41:27.118967: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-18 23:41:

In [21]:
print("Validation dataset:")
(
    y_pred_val,
    non_overlap_patritions_f1_scores_val,
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(
    float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite"
)

print("\nTest dataset:")
(
    y_pred_test,
    non_overlap_patritions_f1_scores_test,
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(
    float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

print("\nTime for Test dataset:")
evaluate_time_of_prediction(
    float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite"
)

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 65.80%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score


/home/polina/newname/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Non-overlap mean F1-score:  0.0

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.0

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 67.12%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score


/home/polina/newname/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Non-overlap mean F1-score:  0.0

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.0

Time for Test dataset:
Accuracy: 67.12%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%

Time to make a prediction for a single data point
Mean: 0.014 seconds
Max: 0.028 seconds
Min: 0.012 seconds


Model file name:  time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float_fallback_q.tflite
File size: 0.085 Megabytes


/home/polina/newname/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
